In [1]:
import os

os.listdir()
os.chdir('../..')
os.listdir()

['LICENSE.md',
 'convokit',
 'requirements.txt',
 'Makefile',
 'website',
 'datasets',
 'tests',
 'README.md',
 'setup.py',
 '.gitignore',
 'CONTRIBUTING.md',
 'examples',
 'doc',
 '.git']

In [2]:
import convokit
from convokit import Corpus, Toxicity, Sequence

corpus = convokit.Corpus(filename="datasets/reddit_coarse_discourse-corpus/corpus")

In [3]:
os.listdir()

['LICENSE.md',
 'convokit',
 'requirements.txt',
 'Makefile',
 'website',
 'datasets',
 'tests',
 'README.md',
 'setup.py',
 '.gitignore',
 'CONTRIBUTING.md',
 'examples',
 'doc',
 '.git']

In [4]:
corpus.print_summary_stats()

Number of Users: 63573
Number of Utterances: 115827
Number of Conversations: 9483


In [5]:
tox = Toxicity(toxicity_json_path="convokit/toxicity/data/reddit_coarse_discourse.json")
tox.transform(corpus)

In [6]:
for convo in corpus.iter_conversations():
    convo_scores = 0
    count = 0

    for utt in convo.iter_utterances():        
        '''
            rerunning this takes over a day for our 110k+ comments since it uses an api with limited query rate, 
            we'll load them from toxicity_dictionary.json that was pre-fetched,
            for others using our transformer, please run self.get_toxicity over the utterances on their corpus.
        '''

        #utt_score = self._get_toxicity(utt.text)


        utt_score = toxicity_scores[utt.id]
        
        convo_scores+=utt_score
        count+=1

        utt.add_meta('toxicity', utt_score)

    convo.add_meta('averagetoxicity', convo_scores/count)

NameError: name 'toxicity_scores' is not defined

In [6]:
list(corpus.iter_utterances())[0].meta['toxicity']

0.21341012

In [11]:
import random

#counter = 0
for convo in corpus.iter_conversations():
    
    temp_chain = []

    for utt in convo.iter_utterances():
        
        if utt.meta['post_depth'] == 2:
            temp_chain.append(utt.id)
    
    if len(temp_chain) > 0:
        
        convo.add_meta('chain', random.choice(temp_chain))
        
        uttid = random.choice(temp_chain)
        chosen_chain= []
        chosen_chain.append(uttid)

        utt = convo.get_utterance(uttid)
        while(utt.meta['post_depth'] > 0):
            if utt.reply_to in corpus.utterances:
                utt = convo.get_utterance(utt.reply_to)
                chosen_chain.append(utt.id)
            else:
                break
                
        #counter+=1
        chosen_chain.reverse()
        convo.add_meta('chain', chosen_chain)
    else:
        convo.add_meta('chain', None)
        
            
print(list(corpus.iter_conversations())[2000])

Conversation({'_owner': <convokit.model.corpus.Corpus object at 0x10845b710>, '_id': 't3_1lddvq', '_utterance_ids': ['t3_1lddvq', 't1_cbye5qd', 't1_cbzrszq', 't1_cbzrvp1', 't1_cbzs1ow', 't1_cbzs2ow', 't1_cc07f3c', 't1_cc0vfqg', 't1_cc0w4ks', 't1_cc0whiu', 't1_cc0xpok'], '_usernames': None, '_meta': {'subreddit': 'showerthoughts', 'url': 'https://www.reddit.com/r/Showerthoughts/comments/1lddvq/what_if_everything_i_do_is_predetermined_and_i/', 'title': 'What if everything I do is predetermined and I have no control over it.', 'averagetoxicity': 0.24006138881818184, 'chain': ['t3_1lddvq', 't1_cbye5qd', 't1_cbzrszq']}})


In [8]:
print(list(corpus.iter_utterances())[2000])

Utterance({'id': 't1_d2h6eld', 'user': User([('name', 'Vaperius')]), 'root': 't3_4gfgf2', 'reply_to': 't1_d2h3fzj', 'timestamp': None, 'text': "You know a country was really trying for world domination when its harder to the pick one of the countries it hasn't had a military conflict with.", 'meta': {'post_depth': 2, 'majority_type': 'elaboration', 'majority_link': 't1_d2h3fzj', 'annotation-types': ['elaboration', 'elaboration', 'humor'], 'annotation-links': ['t1_d2h3fzj', 't1_d2h3fzj', 't1_d2h3fzj'], 'ups': 3, 'toxicity': 0.1813954}})


In [9]:
from collections import defaultdict

upstox_25 = {}
ups_25 = defaultdict(list)
tox_25 = defaultdict(list)

for x in in_top_25:
    if not utts[x].meta['ups'] == None:
        upstox_25[x +"." + convos[x].meta["subreddit"]] = [utts[x].meta['ups'], tox[x]]
        ups_25[convos[x].meta["subreddit"]].append(utts[x].meta['ups'])
        tox_25[convos[x].meta["subreddit"]].append(tox[x])
    else:
        upstox_25[x +"." + convos[x].meta["subreddit"]] = [0, tox[x]]
        ups_25[convos[x].meta["subreddit"]].append(0)
        tox_25[convos[x].meta["subreddit"]].append(tox[x])
corr_25 = {}

for x in ups_25:
    corr_25[x] = pearsonr(tox_25[x],ups_25[x])[0]
    
sorted_corr = sorted(corr_25.items(), key=lambda kv: kv[1])
for x in sorted_corr:
    print(x)

NameError: name 'in_top_25' is not defined

In [14]:
import collections

reddit_corpus = corpus
subreddit_counter = collections.Counter()
convos = reddit_corpus.conversations


for convoutt in convos: 
    #print(convos[convoutt])
    subrd = convos[convoutt].meta["subreddit"]
    subreddit_counter.update({subrd:1})
top_25_subreddits = dict(subreddit_counter.most_common(5))

print(top_25_subreddits.keys())

dict_keys(['askreddit', 'leagueoflegends', 'buildapc', 'explainlikeimfive', 'trees'])


In [58]:
#top sequences
import pandas as pd
from convokit import Corpus, Toxicity, Sequence


convos = corpus.iter_conversations()
utterance_corpus = corpus.utterances

sequences = []

def form_chain(convochain):
    chain = []
    for utt in convochain:
        if utterance_corpus[utt].meta['majority_type']:
            chain.append(utterance_corpus[utt].meta['majority_type'])
        else:
            chain.append("none")
    sequence = '-'.join(chain)
    
    return sequence


for convo in convos:
    if convo.meta['chain']:
        sequences.append(form_chain(convo.meta['chain']))
        


from collections import Counter
sequence_types = Counter(sequences)


top_sequences = sequence_types.most_common(10)

header =["sequences", "count"]

df = pd.DataFrame(top_sequences, columns=["sequences", "count"]) 

print(df)

                              sequences  count
0           question-answer-elaboration   1353
1          question-answer-appreciation   1076
2              question-answer-question    865
3              question-question-answer    587
4             question-answer-agreement    515
5                  question-answer-none    507
6          announcement-question-answer    340
7          question-answer-disagreement    280
8  announcement-elaboration-elaboration    219
9                question-answer-answer    135


### Sequence Transformer

In [11]:
sequence = Sequence()
sequence.transform(corpus)

In [76]:
from statistics import mean 
from IPython.display import display as disp

top_sequence_toxicity = [sequence[0] for sequence in top_sequences]
top_seq_average = []
top_average = []
top_delta = []

top_toxicity_delta = []

convos = reddit_corpus.conversations
print(top_sequence_toxicity)

for sequence in top_sequence_toxicity:
    print(sequence)
    chain_tox = [[], [], []]

    for convo in convos:
        #print(convo)
        
        if convos[convo].meta['chain']:
            chain = form_chain(convos[convo].meta['chain'])
            
            if(chain == sequence):
                for i in range(len(convos[convo].meta["chain_tox"])):
                    chain_tox[i].append(convos[convo].meta["chain_tox"][i])
    
    top_seq_average.append([mean(chain_tox[0]), mean(chain_tox[1]), mean(chain_tox[2])])
    top_average.append(mean([mean(chain_tox[0]), mean(chain_tox[1]), mean(chain_tox[2])]))
    top_delta.append(mean(chain_tox[2])-mean(chain_tox[0]))
    
print(top_average)
print(top_seq_average[0])

#top_sequence_toxicity = [sequence[] for sequence in top_sequences]

header =["sequences", "count"]
sequence_tox_index = list(zip([sequence[0] for sequence in top_sequences],[sequence[1] for sequence in top_sequences], top_average, top_delta, top_seq_average))

df = pd.DataFrame(sequence_tox_index, columns=["sequences", "count", "Avg Toxicity", "Delta", "Sequence Avg"]) 

disp(df)
            
            

['question-answer-elaboration', 'question-answer-appreciation', 'question-answer-question', 'question-question-answer', 'question-answer-agreement', 'question-answer-none', 'announcement-question-answer', 'question-answer-disagreement', 'announcement-elaboration-elaboration', 'question-answer-answer']
question-answer-elaboration
question-answer-appreciation
question-answer-question
question-question-answer
question-answer-agreement
question-answer-none
announcement-question-answer
question-answer-disagreement
announcement-elaboration-elaboration
question-answer-answer
[0.16455091322498153, 0.13420257153268278, 0.14632982605105974, 0.13739298548960818, 0.17180894262394822, 0.17908509574687706, 0.14383926153039214, 0.18512027274369047, 0.18781922903805176, 0.17251559898962965]
[0.1427854205518847, 0.18133224759238728, 0.16953507153067257]


,sequences,count,Avg Toxicity,Delta,Sequence Avg
0,question-answer-elaboration,1353,0.164551,0.026750,"[0.1427854205518847, 0.18133224759238728, 0.16..."
1,question-answer-appreciation,1076,0.134203,-0.038401,"[0.13963546740817845, 0.1617380933866171, 0.10..."
2,question-answer-question,865,0.146330,0.004416,"[0.13370161387283236, 0.16716999279710984, 0.1..."
3,question-question-answer,587,0.137393,-0.012457,"[0.14261336664514482, 0.13940920582708688, 0.1..."
4,question-answer-agreement,515,0.171809,-0.007310,"[0.16237074276116506, 0.19799581710873787, 0.1..."
5,question-answer-none,507,0.179085,0.030705,"[0.15022276366074952, 0.20610496595069033, 0.1..."
6,announcement-question-answer,340,0.143839,0.015522,"[0.1381823749382353, 0.13963070647647058, 0.15..."
7,question-answer-disagreement,280,0.185120,0.031706,"[0.163173864475, 0.1973071993667857, 0.1948797..."
8,announcement-elaboration-elaboration,219,0.187819,0.019937,"[0.16471198684474886, 0.2140964897214612, 0.18..."
9,question-answer-answer,135,0.172516,-0.065220,"[0.20754900531111112, 0.16766838375037038, 0.1..."
